In [12]:
import sys; sys.path.append('..')
import json
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.externals import joblib

from utils.metrics import threshold_search

In [3]:
INPUT_DIR = Path.cwd().joinpath('../../input')
DATA_DIR = Path.cwd().joinpath('../../data')
BQ_PROJECT_ID = json.load(Path.cwd().joinpath('../config.json').open('r'))['bq_project_id']
BQ_DATASET = json.load(Path.cwd().joinpath('../config.json').open('r'))['bq_dataset']

# Metrics Analysis

In [7]:
def load_bq_metrics_tables():
    df_total_metrics = pd.read_gbq(f'select * from {BQ_DATASET + ".total_metrics"}', project_id=BQ_PROJECT_ID)
    df_fold_metrics = pd.read_gbq(f'select * from {BQ_DATASET + ".fold_metrics"}', project_id=BQ_PROJECT_ID)
    df_snapshot_metrics = pd.read_gbq(f'select * from {BQ_DATASET + ".snapshot_metrics"}', project_id=BQ_PROJECT_ID)
    return df_total_metrics, df_fold_metrics, df_snapshot_metrics

In [8]:
df_total_metrics, _, _ = load_bq_metrics_tables()

In [10]:
df_total_metrics.sort_values(by='focal_loss_avg', ascending=True).head(10)

,date,script_name,spec_id,model_name,mv_f1_avg,mv_f1_std,mv_precision_avg,mv_recall_avg,opt_f1_avg,opt_f1_std,...,reopt_precision_avg,reopt_recall_avg,focal_loss_avg,mv_threshold_avg_snapshot_0,mv_threshold_avg_snapshot_1,mv_threshold_avg_snapshot_2,mv_threshold_avg_snapshot_3,mv_threshold_avg_snapshot_4,opt_threshold_avg,reopt_threshold_avg
16,2019-01-26 09:47:38.828937,049_nlpdeeprnn_architecture_search_corrected,7,NLPFeaturesDeepRNN_specId=7,0.691910,0.003475,0.651816,0.737310,0.692137,0.003670,...,0.649554,0.742260,2.267525,0.364,0.362,0.374,0.376,0.372,0.3696,0.368
31,2019-01-26 08:51:59.657296,049_nlpdeeprnn_architecture_search_corrected,6,NLPFeaturesDeepRNN_specId=6,0.692260,0.004366,0.654990,0.734167,0.692202,0.003828,...,0.650912,0.740045,2.361705,0.354,0.362,0.378,0.370,0.356,0.3640,0.364
28,2019-01-25 21:59:57.100308,048_nlprnn_best_embeddings_10fold,0,NLPFeaturesRNN_specId=0_embedtype=glovexfastte...,0.695736,0.004698,0.654076,0.743151,0.695710,0.004762,...,0.651859,0.747853,2.942285,0.364,0.369,0.368,0.372,0.377,0.3700,0.363
4,2019-01-26 01:16:15.599923,048_nlprnn_best_embeddings_10fold,2,NLPFeaturesRNN_specId=2_embedtype=glove+fastte...,0.696514,0.004539,0.655218,0.743460,0.696546,0.004389,...,0.651934,0.749771,2.950367,0.362,0.371,0.375,0.359,0.379,0.3692,0.359
11,2019-01-26 12:54:52.091846,050_nlprnn_embedding_search_2_10fold,0,NLPFeaturesDeepRNN_specId=0,0.697755,0.004031,0.655591,0.745861,0.697283,0.004562,...,0.653092,0.750340,2.955565,0.365,0.372,0.377,0.361,0.372,0.3694,0.363
3,2019-01-25 03:05:38.459553,046_nlprnn_embedding_search,9,NLPFeaturesRNN_specId=9_embedtype=glove+fastte...,0.694522,0.003221,0.654209,0.740156,0.694953,0.003442,...,0.657040,0.738151,3.006108,0.350,0.374,0.388,0.384,0.360,0.3712,0.376
23,2019-01-24 21:00:33.607418,046_nlprnn_embedding_search,3,NLPFeaturesRNN_specId=3_embedtype=glovexparagr...,0.693409,0.002304,0.653721,0.738312,0.693629,0.003180,...,0.653042,0.740416,3.007787,0.360,0.370,0.382,0.382,0.384,0.3756,0.372
32,2019-01-26 03:05:06.949274,048_nlprnn_best_embeddings_10fold,3,NLPFeaturesRNN_specId=3_embedtype=glove+fastte...,0.696178,0.005847,0.657194,0.740193,0.696143,0.005966,...,0.655524,0.744524,3.009023,0.366,0.368,0.377,0.375,0.370,0.3712,0.365
25,2019-01-25 00:55:15.611694,046_nlprnn_embedding_search,7,NLPFeaturesRNN_specId=7_embedtype=glovexfastte...,0.693849,0.002880,0.654643,0.738151,0.693842,0.002991,...,0.655806,0.737520,3.021425,0.358,0.378,0.376,0.374,0.362,0.3696,0.372
5,2019-01-24 19:13:17.552167,046_nlprnn_embedding_search,1,NLPFeaturesRNN_specId=1_embedtype=glovexfastte...,0.691719,0.003127,0.653152,0.735268,0.691193,0.002910,...,0.647789,0.742173,3.036053,0.376,0.354,0.364,0.378,0.370,0.3684,0.356


# Prediction Analysis